In [ ]:
import numpy as np
import scipy.stats as sps
import scipy.optimize as spo
import matplotlib.pyplot as plt
import corner
import sqlite3

In [ ]:
from justice import simulate as sim
from justice import summarize as summ
from justice import visualize as vis
from justice import xform
from justice import lightcurve

In [ ]:
path_to_db = "data/plasticc/data.db"
conn = sqlite3.connect(path_to_db)

In [ ]:
lcs = lightcurve.PlasticcDatasetLC.get_lc_by_target(conn, 42)

In [ ]:
fig = vis.plot_lcs(lcs[2])

In [ ]:
lcs[2].bands['g'].detected